# Project: <project_name>
## sample subset: <primer_set>
## analysis date: <analysis_date>
## user: <name>

### Description
<High-level description of samples, primers, and references used>

### General steps
- prep fastq files
- fastqc on raw fastq files
- clean fastq files using seqyclean
- fastqc on filtered fastq files
- align using bwa
- trim primers using samtools ampliconclip
- generate consensus sequence with ivar consensus

In [ ]:
import os
import glob
import re
import subprocess

import pandas as pd
from Bio import SeqIO

pd.set_option('display.max_rows', 1000)

DOCKER_BASE = 'docker run --rm -v ${PWD}:/data -u $(id -u):$(id -g)'
FASTQC_DOCKER = f'{DOCKER_BASE} staphb/fastqc:0.12.1'
MULTIQC_DOCKER = f'{DOCKER_BASE} staphb/multiqc:1.8'
SEQYCLEAN_DOCKER = f'{DOCKER_BASE} staphb/seqyclean:1.10.09'
BWA_DOCKER = f'{DOCKER_BASE} staphb/bwa:0.7.17'
SAMTOOLS_DOCKER = f'{DOCKER_BASE} staphb/samtools:1.20'
IVAR_DOCKER = f'{DOCKER_BASE} staphb/ivar:1.4.2'

# General Set Up

Edit the below input variables as needed

In [ ]:
# working directory absolute path 
wd = ''  # e.g. /home/sam_baird/flu/h5/test_h5_0005_nextseq/avrl_h5n1_250bp/

project_name = '' # e.g. test_h5_0005_nextseq

### INPUT FILES ###
# just provide the filenames, no paths

# if multiple primer sets in single project, use subworkbook split by primer_set
workbook = '' # e.g. test_h5_0005_nextseq_workbook_avrl_h5n1_250bp.tsv

seqyclean_contaminants_fasta = ''

primer_bed = ''

# keys: short name (used for naming subdirectories and files)
# values: filename
# references can be either single gene segments or multifastas
reference_sequences = {  # e.g. 'h5n1_bovine': 'A_Bovine_texas_029328-01_UtoT.fasta'
    '' : '',
}

# Set up directories and paths

In [ ]:
os.chdir(wd)
refs_dir = 'references'
if not os.path.exists(refs_dir):
    os.mkdir(refs_dir)
reference_sequence_paths = {k: os.path.join(refs_dir, v) for k, v in reference_sequences.items()}

fastq_dir = 'fastq_files'
if not os.path.exists(fastq_dir):
    os.mkdir(fastq_dir)

raw_alignment_dir = 'bwa_alignment'
trimmed_alignment_dir = 'bwa_alignment_trimmed'
primer_bed_path = os.path.join(refs_dir, primer_bed)
fastqc_raw_dir = 'fastqc_raw'
multiqc_raw_dir = 'multiqc_raw'
multiqc_clean_dir = 'multiqc_clean'
seqyclean_dir = 'seqyclean'
fastqc_clean_dir = 'fastqc_clean'
alignment_dir = 'bwa_alignment'
aln_metrics_dir = 'alignment_metrics'
consensus_dir = 'consensus_sequences'
amplicon_dir = 'amplicon_metrics'

**Manual steps needed**: In the working directory, add the:
- workbook
- FASTQ files to the `fastq_files` directory
- reference genomes to the `references` directory
- seqyclean contaminants FASTA to the `references` directory
- primer BED file to the `references` directory

# Read workbook

In [ ]:
path_wb = os.path.join(wd, workbook)
wb = pd.read_csv(path_wb, sep = '\t')

sample_names = wb.sample_name.tolist()
wb

# FastQC on raw FASTQ files
Assess the quality of the raw FASTQ files

In [ ]:
if not os.path.exists(fastqc_raw_dir):
    os.mkdir(fastqc_raw_dir)

for sample in sample_names:
    print(sample)
    
    fastq_1 = f'{sample}_R1.fastq.gz'
    fastq_2 = f'{sample}_R2.fastq.gz'

    outdir = os.path.join(fastqc_raw_dir, sample)
    if not os.path.exists(outdir):
        os.mkdir(outdir)
        
    !{FASTQC_DOCKER} fastqc --outdir fastqc_raw/{sample} fastq_files/{fastq_1}
    !{FASTQC_DOCKER} fastqc --outdir fastqc_raw/{sample} fastq_files/{fastq_2}
    print()

# MultiQC on raw FASTQ files
Combine FastQC results into a single report

In [ ]:
if not os.path.exists(multiqc_raw_dir):
    os.mkdir(multiqc_raw_dir)

!{MULTIQC_DOCKER} multiqc fastqc_raw --outdir {multiqc_raw_dir}

# CSV report on raw FASTQ files

In [ ]:
def summarize_reads(sample_names, fastqc_dir):
    for sample in sample_names:
        print(sample)
        # get fastqc path for zip files
        zip1 = glob.glob(os.path.join(fastqc_dir, sample, f'{sample}*1_fastqc.zip'))[0]
        zip2 = glob.glob(os.path.join(fastqc_dir, sample, f'{sample}*2_fastqc.zip'))[0]
        
        # set unzip outpath
        unzip_outpath = os.path.join(fastqc_dir, sample)
        
        # unzip
        !unzip -q -o {zip1} -d {unzip_outpath}
        !unzip -q -o {zip2} -d {unzip_outpath}
        
        # read in file and create summary out file
        # set up dataframe
        df = pd.DataFrame()
        df['sample_name'] = [sample]
        
        # R1
        path = glob.glob(os.path.join(unzip_outpath, f'{sample}*_fastqc', 'fastqc_data.txt'))[0]
        
        command = f'cat {path} | grep "Total Sequences" | cut -f 2'
        total_seqs = subprocess.check_output(command, shell=True, text=True).rstrip('\n')
        df['r1_total_reads'] = [total_seqs]

        command = f'cat {path} | grep "Sequences flagged as poor quality" | cut -f 2'
        flagged_seqs = subprocess.check_output(command, shell=True, text=True).rstrip('\n')
        df['r1_flagged_reads_as_poor_quality'] = [flagged_seqs]

        command = f'cat {path} | grep "Sequence length" | cut -f 2'
        seq_len = subprocess.check_output(command, shell=True, text=True).rstrip('\n')
        df['r1_read_len'] = [seq_len]
        
        # R2
        path = glob.glob(os.path.join(unzip_outpath, f'{sample}*2_fastqc', 'fastqc_data.txt'))[0]
        
        command = f'cat {path} | grep "Total Sequences" | cut -f 2'
        total_seqs = subprocess.check_output(command, shell = True, text = True).rstrip('\n')
        df['r2_total_reads'] = [total_seqs]

        command = f'cat {path} | grep "Sequences flagged as poor quality" | cut -f 2'
        flagged_seqs = subprocess.check_output(command, shell = True, text = True).rstrip('\n')
        df['r2_flagged_reads_as_poor_quality'] = [flagged_seqs]

        command = f'cat {path} | grep "Sequence length" | cut -f 2'
        seq_len = subprocess.check_output(command, shell = True, text = True).rstrip('\n')
        df['r2_read_len'] = [seq_len]
        
        outfile = os.path.join(unzip_outpath, f'{sample}_summary_metrics.tsv')
        df.to_csv(outfile, sep = '\t', index = False)

summarize_reads(sample_names, fastqc_raw_dir)

# Seqyclean on raw FASTQ files
Filter by quality and remove known contaminants

In [ ]:
if not os.path.exists(seqyclean_dir):
    os.mkdir(seqyclean_dir)

seqyclean_commands = []
for sample in sample_names:    
    fastq_1 = f'fastq_files/{sample}_R1.fastq.gz'
    fastq_2 = f'fastq_files/{sample}_R2.fastq.gz'         
    out_name = f'seqyclean/{sample}_clean'

    !{SEQYCLEAN_DOCKER} seqyclean -minlen 25 -qual 30 30 -gz -1 {fastq_1} -2 {fastq_2} -c references/Adapters_plus_PhiX_174.fasta -o {out_name}

# FastQC on cleaned FASTQ files

In [ ]:
if not os.path.exists(fastqc_clean_dir):
    os.mkdir(fastqc_clean_dir)

for sample in sample_names:
    print(sample)
    
    fastq_1 = f'{sample}_clean_PE1.fastq.gz'
    fastq_2 = f'{sample}_clean_PE2.fastq.gz'

    outdir = os.path.join(fastqc_clean_dir, sample)
    if not os.path.exists(outdir):
        os.mkdir(outdir)
                
    !{FASTQC_DOCKER} fastqc --outdir fastqc_clean/{sample} seqyclean/{fastq_1}
    !{FASTQC_DOCKER} fastqc --outdir fastqc_clean/{sample} seqyclean/{fastq_2}
    print()

# MultiQC on cleaned FASTQ files

In [ ]:
if not os.path.exists(multiqc_clean_dir):
    os.mkdir(multiqc_clean_dir)

!{MULTIQC_DOCKER} multiqc fastqc_clean --outdir {multiqc_clean_dir}

# CSV report on raw FASTQ files

In [ ]:
clean_reads_summary = summarize_reads(sample_names, fastqc_clean_dir)
clean_reads_summary

# Get summary metrics on reads

In [ ]:
# combine fastqc sumary from raw and filtered data
df_raw_list = []
df_filtered_list = []
    
    
for sample in sample_names:    
    # first read in raw summary file
    path = os.path.join(wd, 'fastqc_raw', sample, f'{sample}_summary_metrics.tsv')
    df = pd.read_csv(path, sep = '\t')
    
    df_raw_list.append(df)
    
    # now read in filtered summary file
    path = os.path.join(wd, 'fastqc_clean', sample, f'{sample}_summary_metrics.tsv')
    df = pd.read_csv(path, sep = '\t')
    
    df_filtered_list.append(df)
    
    
# concatenate dataframes and join
raw_df = pd.concat(df_raw_list).reset_index(drop = True)
raw_df = raw_df.set_index('sample_name')

filtered_df = pd.concat(df_filtered_list).reset_index(drop = True)
filtered_df = filtered_df.set_index('sample_name')


combined_df = raw_df.join(filtered_df, how = 'left', lsuffix = '_raw', rsuffix = '_filtered')
combined_df = combined_df.reset_index()

# calculate total reads diff
combined_df['total_reads_diff'] = combined_df['r1_total_reads_raw'] - combined_df['r1_total_reads_filtered']
combined_df['raw_total_reads_paired'] = combined_df['r1_total_reads_raw']
combined_df['filtered_total_reads_paired'] = combined_df['r1_total_reads_filtered']

# add primer set
for row in range(combined_df.shape[0]):
    sample = combined_df.sample_name[row]

# print(j.columns)

col_order = ['sample_name', 'raw_total_reads_paired', 'filtered_total_reads_paired',
             'total_reads_diff', 'r1_total_reads_raw', 'r1_flagged_reads_as_poor_quality_raw',
             'r1_read_len_raw', 'r2_total_reads_raw', 'r2_flagged_reads_as_poor_quality_raw',
             'r2_read_len_raw', 'r1_total_reads_filtered', 'r1_flagged_reads_as_poor_quality_filtered',
             'r1_read_len_filtered', 'r2_total_reads_filtered',
             'r2_flagged_reads_as_poor_quality_filtered', 'r2_read_len_filtered']

combined_df = combined_df[col_order]
combined_df['project_name'] = project_name

outfile = os.path.join(wd, f'{project_name}_reads_QC_summary.csv')
combined_df.to_csv(outfile, index = False)
combined_df

combined_df

# Align to each reference using BWA

In [ ]:
if not os.path.exists(alignment_dir):
    os.mkdir(alignment_dir)

for sample in sample_names:
    # align each sample to each reference
    for ref in reference_sequence_paths:

        fastq_1 = os.path.join(seqyclean_dir, f'{sample}_clean_PE1.fastq.gz')
        fastq_2 = os.path.join(seqyclean_dir, f'{sample}_clean_PE2.fastq.gz')

        ref_base_name = ref.split('/')[-1]

        # each reference gets its own subdirectory
        out_subdir = os.path.join(alignment_dir, ref_base_name)
        if not os.path.exists(out_subdir):
            os.mkdir(out_subdir)

        outsam = os.path.join(out_subdir, f'{sample}.sam')
        outbam = os.path.join(out_subdir, f'{sample}.aln.sorted.bam')

        !{BWA_DOCKER} bwa index -p {ref_base_name} -a is {reference_sequence_paths[ref]}
        !{BWA_DOCKER} bwa mem -t 6 {ref_base_name} {fastq_1} {fastq_2} -f {outsam}
        !{SAMTOOLS_DOCKER} samtools view -bS {outsam} | samtools sort -o {outbam}
        !rm {outsam}  # save storage space

# Create a sorted primer BED file for samtools ampliconclip/ampliconstats 

In [ ]:
bed_df = pd.read_csv(primer_bed_path, sep='\t', header=None)

def extract_amplicon_name(primer_name):
    last_separator = max(primer_name.rfind('-'), primer_name.rfind('_'))
    if last_separator != -1:
        amplicon = primer_name[:last_separator]
    else:
        raise ValueError(f'Could not parse amplicon name of primer {primer_name}')
    return amplicon

bed_df['amplicon'] = bed_df[3].apply(extract_amplicon_name)

# amplicon number assumed to be last string of digits in amplicon_name
bed_df['amplicon_number'] = bed_df[3].str.extract(r'(\d+)(?!.*\d)', expand=False)
bed_df['amplicon_number'] = bed_df['amplicon_number'].astype(int)

bed_df = bed_df.sort_values([0, 'amplicon_number', 5])

# check that the BAM reference name(s) can be matched to reference(s) in the BED
# In the WDL, we probably want to either error out or not output a BAM in trim step
bed_references = set(bed_df[0])
for bam in glob.glob(f'{alignment_dir}/*/*'):
    bam_sq_header = !{SAMTOOLS_DOCKER} samtools view -H {bam} | grep '^@SQ'
    bam_references = set()
    for line in bam_sq_header:
        bam_references.add(line.split('\t')[1].removeprefix('SN:'))
    missing = bam_references - bed_references
    if missing:
        print(f'WARNING: The reference(s) {missing} from BAM could not be matched to a corresponding reference in BED for {bam}.\n')

sorted_bed_path = os.path.join(refs_dir, f'sorted_{primer_bed}')
bed_df.drop(columns=['amplicon', 'amplicon_number']).to_csv(sorted_bed_path, sep='\t', header=False, index=False)
bed_df

# Trim primers from alignment using samtools ampliconclip

In [ ]:
for sample in sample_names:
    for ref in reference_sequence_paths:

        out_subdir = os.path.join(trimmed_alignment_dir, ref)
        os.makedirs(out_subdir, exist_ok=True)

        in_subdir = os.path.join(raw_alignment_dir, ref)
        aln_bam = os.path.join(in_subdir, f'{sample}.aln.sorted.bam')

        trimmed_bam = os.path.join(out_subdir, f'{sample}_trimmed.bam')
        trimmed_sorted_bam = os.path.join(out_subdir, f'{sample}_trimmed.sorted.bam')

        # we use --both-ends since small enough fragments will sequence into the opposite primer
        # first column in BED must match reference name(s) in BAM for primers to be trimmed
        !{SAMTOOLS_DOCKER} samtools ampliconclip --both-ends -b {sorted_bed_path} -o {trimmed_bam} {aln_bam}

        !{SAMTOOLS_DOCKER} samtools sort {trimmed_bam} -o {trimmed_sorted_bam}
        !{SAMTOOLS_DOCKER} samtools index {trimmed_sorted_bam}
        !rm {trimmed_bam}


# Calculate alignment metrics

In [ ]:
if not os.path.exists(aln_metrics_dir):
    os.mkdir(aln_metrics_dir)
    
for sample in sample_names:
    for ref in reference_sequence_paths:
        aln_dir = os.path.join('bwa_alignment_trimmed', ref)
        bam_file = os.path.join(aln_dir, f'{sample}_trimmed.sorted.bam')

        out_subdir_aln_metrics = os.path.join(aln_metrics_dir, ref)
        if not os.path.exists(out_subdir_aln_metrics):
            os.mkdir(out_subdir_aln_metrics)

        out_bam_coverage = os.path.join(out_subdir_aln_metrics, f'{sample}_{ref}_coverage.txt')
        out_bam_stats = os.path.join(out_subdir_aln_metrics, f'{sample}_{ref}_stats.txt')

        !{SAMTOOLS_DOCKER} samtools coverage -o {out_bam_coverage} {bam_file}
        !{SAMTOOLS_DOCKER} sh -c "samtools stats {bam_file} > {out_bam_stats}"

        # Output per-segment stats for multi-segment references
        segments = !{SAMTOOLS_DOCKER} samtools idxstats {bam_file} | cut -f1 | grep -v '*'
        if len(segments) > 1:
            for segment in segments:
                out_bam_coverage = os.path.join(out_subdir_aln_metrics,
                                                segment,
                                                f'{sample}_{ref}_{segment}_coverage.txt')

                out_bam_stats = os.path.join(out_subdir_aln_metrics,
                                            segment,
                                            f'{sample}_{ref}_{segment}_stats.txt')

                !{SAMTOOLS_DOCKER} samtools coverage --region {segment} -o {out_bam_coverage} {bam_file}
                !{SAMTOOLS_DOCKER} sh -c "samtools stats {bam_file} {segment} > {out_bam_stats}"

# Determine consensus sequence using ivar consensus

In [ ]:
if not os.path.exists(consensus_dir):
    os.mkdir(consensus_dir)

for sample in sample_names:
    for ref in reference_sequence_paths:
        out_subdir_consensus = os.path.join(consensus_dir, ref)
        if not os.path.exists(out_subdir_consensus):
            os.mkdir(out_subdir_consensus)

        trimmed_sorted_bam = os.path.join(trimmed_alignment_dir, ref, f'{sample}_trimmed.sorted.bam')
        pileup_txt = os.path.join(trimmed_alignment_dir, ref, f'{sample}_pileup.txt')

        # generate pileup
        !{SAMTOOLS_DOCKER} samtools faidx {reference_sequence_paths[ref]}
        !{SAMTOOLS_DOCKER} samtools mpileup -A -aa -d 600000 -B -Q 20 -q 20 -f {reference_sequence_paths[ref]} {trimmed_sorted_bam} -o {pileup_txt}

        records = SeqIO.parse(reference_sequence_paths[ref], format='fasta')
        num_records = len(list(records))
        # if reference is a multifasta, write separate consensus for each gene segment
        if num_records > 1:
            segment_fastas = []
            for segment in SeqIO.parse(reference_sequence_paths[ref], format='fasta'):
                out_subdir_consensus_segment = os.path.join(consensus_dir, ref, segment.id)
                os.makedirs(out_subdir_consensus_segment, exist_ok=True)
                consensus_prefix = os.path.join(out_subdir_consensus_segment, f'{sample}_{ref}_{segment.id}_consensus')

                command = f"cat {pileup_txt} | grep {segment.id} | ivar consensus -p {consensus_prefix} -q 20 -t 0.6 -m 10"
                print(command)
                !{IVAR_DOCKER} sh -c "{command}"
                
                segment_fasta = f'{consensus_prefix}.fa'
                segment_fastas.append(segment_fasta)
                
            # also output a fasta with all the gene segments
            segments_fastas_str = ' '.join(segment_fastas)
            !cat {segments_fastas_str} > {out_subdir_consensus}/{sample}_{ref}_consensus.fa
        else:
            out_subdir_consensus = os.path.join(consensus_dir, ref)
            consensus_prefix = os.path.join(out_subdir_consensus, f'{sample}_{ref}_consensus')
            !{IVAR_DOCKER} sh -c "cat {pileup_txt} | ivar consensus -p {consensus_prefix} -q 20 -t 0.6 -m 10"

# Calculate percent coverage

In [ ]:
# reference length with and without primers
reference_lengths = {}
for ref in reference_sequence_paths:
    records = list(SeqIO.parse(reference_sequence_paths[ref], format='fasta'))
    num_records = len(records)
    if num_records > 1:
        segment_lengths = {}
        for segment in records:
            total_length = len(segment.seq)
            segment_bed_df = bed_df[bed_df[0] == segment.id]

#                       Primer0                      PrimerN
#                       ======>                    <=========
#             sequence: A  T  C  G  G  C  G  A  T  T  A  A  A 
#             0-based:  0  1  2  3  4  5  6  7  8  9  10 11 12
#             1-based:  1  2  3  4  5  6  7  8  9  10 11 12 13

#             Primer1 BED coordinates: [0,3)
#             PrimerN BED coordinates: [9,13)
#             Insert length: 9 - 3 = 6

            primer_insert_length = segment_bed_df[1].max() - segment_bed_df[2].min()

            segment_lengths[segment.id] = (total_length, primer_insert_length)
        reference_lengths[ref] = segment_lengths
    else:
        record = records[0]
        total_length = len(record.seq)
        segment_bed_df = bed_df[bed_df[0] == record.id]
        if segment_bed_df.empty:
            raise ValueError(f'Reference {record.id} not found in BED')
        primer_insert_length = segment_bed_df[1].max() - segment_bed_df[2].min()
        reference_lengths[ref] = (total_length, primer_insert_length)

reference_lengths

In [ ]:
def calculate_percent_coverage(sample, consensus, total_reference_length, primer_insert_length):
    record = SeqIO.read(consensus, 'fasta')

    seq_len = len(record.seq)
    number_ns = record.seq.count('N')
    total_seq_len = seq_len - number_ns

    # count N's in front of sequence and N's at back of sequence
    seq_str = str(record.seq)

    if re.search( '^N+', seq_str):
        string = re.findall('^N+', seq_str)[0]
        front_ns = len(string)
    else:
        front_ns = 0

    if re.search( 'N+$', seq_str):
        string = re.findall('N+$', seq_str)[0]
        back_ns = len(string)
    else:
        back_ns = 0

    percent_coverage_total_length = round(total_seq_len/total_reference_length * 100,1)
    percent_coverage_primer_insert_length = round(total_seq_len/primer_insert_length * 100,1)
    
    consensus_path_split = consensus.split('/')
    if len(consensus_path_split) == 4:
        reference = f'{consensus_path_split[1]}: {consensus_path_split[2]}'
        subdir = os.path.join(consensus_path_split[1], consensus_path_split[2])
    else:
        reference = consensus_path_split[1]
        subdir = consensus_path_split[1]

    df = pd.DataFrame()
    df['sample_name'] = [sample]
    df['reference'] = reference
    df['percent_coverage_total_length'] = percent_coverage_total_length
    df['percent_coverage_primer_insert_length'] = percent_coverage_primer_insert_length
    df['number_aln_bases'] = seq_len
    df['ambiguous_bases'] = number_ns
    df['number_aln_nonambiguous_bases'] = total_seq_len
    df['front_ns'] = front_ns
    df['back_ns'] = back_ns
    
    os.makedirs(os.path.join(aln_metrics_dir, subdir), exist_ok=True)
    fname = f'{consensus_path_split[-1].removesuffix("_consensus.fa")}_percent_coverage.csv'
    outfile = os.path.join(aln_metrics_dir, subdir, fname)
    df.to_csv(outfile, index=False)

    return df

percent_coverage_dfs = []
for sample in sample_names:
    for ref in reference_lengths:
        # if reference is multi-segment
        if isinstance(reference_lengths[ref], dict):
            for segment in reference_lengths[ref]:
                consensus = os.path.join(consensus_dir, ref, segment, f'{sample}_{ref}_{segment}_consensus.fa')
                percent_coverage_df = calculate_percent_coverage(
                    sample,
                    consensus,
                    reference_lengths[ref][segment][0],
                    reference_lengths[ref][segment][1],
                )
                percent_coverage_dfs.append(percent_coverage_df)
        
        else:
            consensus = os.path.join(consensus_dir, ref, f'{sample}_{ref}_consensus.fa')
            percent_coverage_df = calculate_percent_coverage(
                sample,
                consensus,
                reference_lengths[ref][0],
                reference_lengths[ref][1],
            )
            percent_coverage_dfs.append(percent_coverage_df)
            
concat_percent_coverage_df = pd.concat(percent_coverage_dfs).reset_index(drop=True)
concat_percent_coverage_outfile = os.path.join(aln_metrics_dir, f'{project_name}_percent_coverage.csv')
concat_percent_coverage_df.to_csv(concat_percent_coverage_outfile, index=False)
concat_percent_coverage_df

In [ ]:
def summarize_coverage(sample, stats_dir):
    stats_dir_split = stats_dir.split('/')
    if len(stats_dir_split) == 3:
        # ref_segment
        ref_str = f'{stats_dir_split[1]}_{stats_dir_split[2]}'
    else:
        # just ref
        ref_str = f'{stats_dir_split[1]}'
    
    bam_coverage = os.path.join(stats_dir, f'{sample}_{ref_str}_coverage.txt')
    bam_cov_df = pd.read_csv(bam_coverage, sep = '\t')

    # pull out number of reads mapped and use with fastqc to calculate 
    # the number of unmapped reads and the percent of reads mapped
    bam_stats = os.path.join(stats_dir, f'{sample}_{ref_str}_stats.txt')
    with open (bam_stats, 'r') as file:
        for line in file:
            if re.search('reads mapped:', line):
                reads_mapped = float(line.split('\t')[2].strip())


    # pull out number of reads from fastqc output
    fastqc_metrics_file = os.path.join(wd, "fastqc_clean", sample, f'{sample}_summary_metrics.tsv')
    fastqc_df = pd.read_csv(fastqc_metrics_file, sep = '\t')
    total_raw_reads = fastqc_df['r1_total_reads'][0] + fastqc_df['r2_total_reads'][0]

    # calcuate the number of unmapped reads and the percent reads mapped
    if total_raw_reads != 0:
        percent_reads_mapped = round(reads_mapped/total_raw_reads * 100, 1)
    else:
        percent_reads_mapped = 0

    reads_unmapped = total_raw_reads - reads_mapped

    # pull out percent coverage 1 and 2 from percent coverage
    percent_coverage_file = os.path.join(stats_dir, f'{sample}_{ref_str}_percent_coverage.csv')
    percent_df = pd.read_csv(percent_coverage_file)


    df = pd.DataFrame()
    df['sample_name'] = [sample]
    df['reference'] = [percent_df['reference'].iloc[0]]

    df['percent_reads_mapped'] = percent_reads_mapped
    df['total_raw_reads'] = total_raw_reads
    df['reads_mapped'] = reads_mapped
    df['reads_unmapped'] = reads_unmapped
    df['av_depth'] = [bam_cov_df['meandepth'].iloc[0]]
    df['meanmapq'] = [bam_cov_df['meanmapq'].iloc[0]]

    df['percent_coverage_total_length'] = [percent_df['percent_coverage_total_length'].iloc[0]]
    df['percent_coverage_primer_insert_length'] = [percent_df['percent_coverage_primer_insert_length'].iloc[0]]
    df['front_ns'] = [percent_df['front_ns'].iloc[0]]
    df['back_ns'] = [percent_df['back_ns'].iloc[0]]
    
    outfile = os.path.join(stats_dir, f'{sample}_{ref_str}_aln_metrics_summary.csv')

    df.to_csv(outfile, index=False)

    return df

coverage_summary_dfs = []
for sample in sample_names:
    for ref in reference_sequence_paths:
        records = SeqIO.parse(reference_sequence_paths[ref], format='fasta')
        num_records = len(list(records))
        if num_records > 1:
            for segment in SeqIO.parse(reference_sequence_paths[ref], format='fasta'):
                stats_dir = os.path.join(aln_metrics_dir, ref, segment.id)
                coverage_summary_df = summarize_coverage(sample, stats_dir)
                coverage_summary_dfs.append(coverage_summary_df)
        else:
            stats_dir = os.path.join(aln_metrics_dir, ref)
            coverage_summary_df = summarize_coverage(sample, stats_dir)
            coverage_summary_dfs.append(coverage_summary_df)

concat_coverage_summary_df = pd.concat(coverage_summary_dfs).reset_index(drop=True)
concat_coverage_summary_outfile = os.path.join(aln_metrics_dir, f'{project_name}_aln_metrics_summary.csv')
concat_coverage_summary_df.to_csv(concat_coverage_summary_outfile, index=False)
concat_coverage_summary_df       

# Calculate per-amplicon coverage

In [ ]:
if not os.path.exists(amplicon_dir):
    os.mkdir(amplicon_dir)

num_primers = bed_df[3].nunique()
amplicons = list(bed_df['amplicon'].unique())
if num_primers / len(amplicons) != 2:
    raise ValueError('Either more than two primers per amplicon or cannot properly parse primer names in BED')

amplicon_min_depths_dfs = []
for sample in sample_names:
    for ref in reference_sequence_paths:
        trimmed_sorted_bam = os.path.join(trimmed_alignment_dir, ref, f'{sample}_trimmed.sorted.bam')
        amplicon_subdir = os.path.join(amplicon_dir, ref)
        if not os.path.exists(amplicon_subdir):
            os.mkdir(amplicon_subdir)
        ampliconstats_out = os.path.join(amplicon_subdir, f'{sample}_{ref}_ampliconstats.txt')
        ampliconstats_plots_dir = os.path.join(amplicon_subdir, f'{sample}_{ref}_ampliconstats_plots')
        os.makedirs(ampliconstats_plots_dir, exist_ok=True)
        plot_ampliconstats_prefix = os.path.join(ampliconstats_plots_dir, f'{sample}_{ref}_ampliconstats')
        
        # calculate coverage stats per amplicon
        !{SAMTOOLS_DOCKER} samtools ampliconstats {sorted_bed_path} {trimmed_sorted_bam} -o {ampliconstats_out}
        !{SAMTOOLS_DOCKER} plot-ampliconstats {plot_ampliconstats_prefix} {ampliconstats_out}

        # only select amplicons relevant to BAM   
        bam_sq_header = !{SAMTOOLS_DOCKER} samtools view -H {trimmed_sorted_bam} | grep '^@SQ'
        bam_references = set()
        for line in bam_sq_header:
            bam_references.add(line.split('\t')[1].removeprefix('SN:'))   
        subset_bed_df = bed_df[bed_df[0].isin(bam_references)]
        amplicons = subset_bed_df['amplicon'].unique().tolist()

        # get depth at every position
        # use -J so that deletions don't get counted as 0 depth
        depths_out = os.path.join(amplicon_subdir, f'{sample}_{ref}_depths.txt')
        !{SAMTOOLS_DOCKER} samtools depth -aJ {trimmed_sorted_bam} -o {depths_out}

        depths_df = pd.read_csv(depths_out, header=None, sep='\t',
                                names=['reference', 'position', 'depth'])

        os.remove(depths_out)

        # determine minimum depth per amplicon
        if not depths_df.empty:
            amplicon_min_depths = []
            for amplicon in amplicons:
                amplicon_record = {}
                amplicon_start = subset_bed_df[(subset_bed_df['amplicon'] == amplicon)
                                               & (subset_bed_df[5] == '+')][2].values[0] + 1

                amplicon_end = subset_bed_df[(subset_bed_df['amplicon'] == amplicon)
                                             & (subset_bed_df[5] == '-')][1].values[0]
                
                current_reference = subset_bed_df[subset_bed_df['amplicon'] == amplicon][0].iloc[0]

                subset_depths_df = depths_df[(depths_df['reference'] == current_reference)
                                             & (depths_df['position'] >= amplicon_start)
                                             & (depths_df['position'] <= amplicon_end)]
                                
                amplicon_min_depth = subset_depths_df['depth'].min()

                amplicon_record['sample_name'] = sample
                amplicon_record['reference'] = ref
                amplicon_record['amplicon'] = amplicon
                amplicon_record['start'] = amplicon_start
                amplicon_record['end'] = amplicon_end
                amplicon_record['min_depth'] = amplicon_min_depth if pd.notna(amplicon_min_depth) else 0
                amplicon_min_depths.append(amplicon_record)

            amplicon_depths_df = pd.DataFrame.from_records(amplicon_min_depths)
            amplicon_min_depths_out = os.path.join(amplicon_subdir, f'{sample}_{ref}_amplicon_min_depths.tsv')
            amplicon_depths_df.to_csv(amplicon_min_depths_out, sep='\t', index=False)
            amplicon_min_depths_dfs.append(amplicon_depths_df)

concat_amplicon_min_depths_df = pd.concat(amplicon_min_depths_dfs).reset_index(drop=True)
concat_amplicon_min_depths_out = os.path.join(amplicon_dir, f'{project_name}_amplicon_min_depths.tsv')
concat_amplicon_min_depths_df.to_csv(concat_amplicon_min_depths_out, sep='\t', index=False)
concat_amplicon_min_depths_df